# Initialization & setup

In [20]:
!pip uninstall tensorflow; pip install tensorflow==2.1.0#2.0.0-beta1

Uninstalling tensorflow-1.15.0:
  Would remove:
    /tensorflow-1.15.0/python3.6/tensorflow-1.15.0.dist-info/*
    /tensorflow-1.15.0/python3.6/tensorflow/*
    /tensorflow-1.15.0/python3.6/tensorflow_core/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-1.15.0
ERROR: Could not find a version that satisfies the requirement tensorflow==2.1.0#2.0.0-beta1 (from versions: 0.12.1, 1.0.0, 1.0.1, 1.1.0rc0, 1.1.0rc1, 1.1.0rc2, 1.1.0, 1.2.0rc0, 1.2.0rc1, 1.2.0rc2, 1.2.0, 1.2.1, 1.3.0rc0, 1.3.0rc1, 1.3.0rc2, 1.3.0, 1.4.0rc0, 1.4.0rc1, 1.4.0, 1.4.1, 1.5.0rc0, 1.5.0rc1, 1.5.0, 1.5.1, 1.6.0rc0, 1.6.0rc1, 1.6.0, 1.7.0rc0, 1.7.0rc1, 1.7.0, 1.7.1, 1.8.0rc0, 1.8.0rc1, 1.8.0, 1.9.0rc0, 1.9.0rc1, 1.9.0rc2, 1.9.0, 1.10.0rc0, 1.10.0rc1, 1.10.0, 1.10.1, 1.11.0rc0, 1.11.0rc1, 1.11.0rc2, 1.11.0, 1.12.0rc0, 1.12.0rc1, 1.12.0rc2, 1.12.0, 1.12.2, 1.12.3, 1.13.0rc0, 1.13.0rc1, 1.13.0rc2, 1.13.1, 1.13.2, 1.14.0rc0, 1.14.0rc1, 1.14.0, 1.15.0rc0, 1.15.0rc1, 1.15.0rc2, 1.15.0rc3, 1.15.0, 1.15.2, 2.0.0a0, 2.0.

In [21]:
#from google.colab import drive

#root = "/content/gdrive"
#drive.mount(root)
import sys
ROOT = '/content/drive/My Drive/'
PROJ = 'Tesi/'
LOG = PROJ + 'logs/file.log'
PROJ_COMPLETE_PATH = ROOT + PROJ
LOG_COMPELTE_PATH = ROOT + LOG

if(PROJ_COMPLETE_PATH not in sys.path):
  sys.path.append(PROJ_COMPLETE_PATH)

if(ROOT not in sys.path):
  sys.path.append(ROOT)

sys.path

['/tensorflow-1.15.0/python3.6',
 '',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/My Drive/Tesi/',
 '/content/drive/My Drive/']

In [0]:
### uncomment these line if there isn't already fuzzylearn repo in google drive
#!rm -rf {PROJ_COMPLETE_PATH}
#!mkdir {PROJ_COMPLETE_PATH}
#!git clone https://tomgeek27@bitbucket.org/tomgeek27/riconoscimento-volti-tesi.git {PROJ_COMPLETE_PATH}

### uncomment this line if you need to update your repo (updated from develop branch)
#!cd {PROJ_COMPLETE_PATH}; git pull origin develop

In [0]:
import uuid

def log_init(folder="/content/drive/My Drive/Tesi/logs/", fname=""):
  assert(fname != "")
  filename = folder + fname
  import logging

  logger = logging.getLogger(__name__)
  logger.setLevel(logging.INFO)
  if(not logger.hasHandlers()):
    f_handler = logging.FileHandler(filename)
    f_handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))

    c_handler = logging.StreamHandler()
    c_handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
    logger.addHandler(f_handler)
    logger.addHandler(c_handler)

  return logger

# Data processing

In [24]:
from sklearn.datasets import fetch_olivetti_faces
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np

olivetti = fetch_olivetti_faces()

olivetti_data = olivetti.data
olivetti_labels = olivetti.target

#olivetti_data_tsne = TSNE(n_components=2).fit_transform(olivetti_data)

#from sklearn.model_selection import train_test_split

#X_train, X_test, y_train, y_test = train_test_split(olivetti_data_tsne, olivetti_labels, stratify=olivetti_labels, random_state=0, test_size=.3)

downloading Olivetti faces from https://ndownloader.figshare.com/files/5976027 to /root/scikit_learn_data


In [0]:
def get_classes(dataset_labels):
  return np.unique(dataset_labels)

def str2fuzzifier(f_str):
  if(f_str == 'LinearFuzzifier'):
    return fl.fuzzifiers.LinearFuzzifier
  elif(f_str == 'QuantileLinearPiecewiseFuzzifier'):
    return fl.fuzzifiers.QuantileLinearPiecewiseFuzzifier

def str2kernel(k_str):
  if(k_str == 'GaussianKernel'):
    return fl.kernel.GaussianKernel

In [33]:
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedShuffleSplit
import fuzzylearn as fl
import logging
from sklearn.model_selection import GroupKFold
import importlib

importlib.reload(fl)
def estimate_membership_holdout(x, y, c=1, k=fl.kernel.GaussianKernel(sigma=.5), fuzzifier=fl.fuzzifiers.LinearFuzzifier, y_needs_mapping=False, label=""):
  if(y_needs_mapping):
    y = map_labels(y, label)
  model = fl.FuzzyInductor(c=c, k=k, fuzzifier=fuzzifier).fit(x, y)

  return model

def grid_search(x, y, params, data_dimension, data_reduction="t", logger=None, y_needs_mapping=False, label="", folds=3):
  check_label(label)
  
  if(y_needs_mapping):
    y = map_labels(y, label)

  log_info(logger, label, "Started GridSearchCV with: %s", params)
  log_info(logger, label, "DATA_DIMENSION_: %s", data_dimension)
  clf = GridSearchCV(fl.FuzzyInductor(), params, cv=stratified_folds(x, y, folds=folds)).fit(x, y)

  log_info(logger, label, "CV_RESULTS_: %s", clf.cv_results_)
  log_info(logger, label, "BEST_SCORE_: %s", clf.best_score_)
  log_info(logger, label, "BEST_PARAMS_: %s", clf.best_params_)
  print(clf.cv_results_)
  print(clf.best_score_)
  print(clf.best_params_)
  return clf


def nested_grid_search(x, y, params, logger=None, y_needs_mapping=False, label="", folds=3, innerFolds=3):
  check_label(label)

  log_info(logger, label, "Started Nested-GridSearchCV with: %s", params)
  if(y_needs_mapping):
    y = map_labels(y, label)

  clf = grid_search(x, y, params, logger=logger, y_needs_mapping=not y_needs_mapping, label=label, folds=innerFolds)
  results = cross_val_score(clf.best_estimator_, x, y, cv=stratified_folds(x, y, folds=folds), verbose=10)

  log_info(logger, label, "NESTED CROSS VALIDATION with %s fold: %s", folds, results)
  log_info(logger, label, "NESTED CROSS VALIDATION (mean): %s", np.mean(results))

def log_info(logger, label, message, *args):
  if(logger != None):
    logger.info("(" + str(label) + ") - " + message, *args)

def map_labels(y, label):
    assert(label != "")
    return [1 if elem == label else 0 for elem in y]

def check_label(label):
  assert(label != "" or label is not None) #is important to know which class is training 

def stratified_folds(X, y, random_state=0, folds=3):
  return StratifiedShuffleSplit(n_splits=folds, random_state=random_state).split(X, y)


gurobi not available


```
estimate_membership_holdout
```

Questa funzione rappresenta è utile a definire una model_selection effettuata tramite un semplice holdout, la quale vede come argomenti tutti fgli argomenti del modello 'FuzzyInductor'

# Logging

Here you can find all tools that you need to retrieve information from _file.log_

In [30]:
def lastAnalyzedClass(lines):
  """Return the index of the last trained class, you could use it for train all model after the last trained"""
  return list(bestParams(lines).keys())[-1]

def bestParams(lines):
  """Return params produced best results"""
  return _searchForValue(lines, "BEST_PARAMS_")

def bestScores(lines):
  """Return best score for all model"""
  return _searchForValue(lines, "BEST_SCORE_")

def params(lines):
  """Return all params used for train all model"""
  return _searchForValue(lines, "Started GridSearchCV")

def dataDimension(lines):
  """Return the number of feature used for training model"""
  dataDimension_ = _searchForValue(lines, "DATA_DIMENSION_")
  if(dataDimension_ == {}):
    defaultMap = {}

    for i in np.unique(olivetti_labels):
      defaultMap[i] = 2
    
    return defaultMap

  return dataDimension_

def _searchForValue(lines, v):
  m = {}
  for x in lines:
    if(v in x):
      c = x[x.index('(') + 1: x.index(')')]
      m[c] = x[x.index(': ')+2:-1]
  
  return m

def _searchForValue2(lines, v):
  m = []
  for x in lines:
    if(v in x):
      c = x[x.index('(') + 1: x.index(')')]
      m.append(x[x.index(': ')+2:-1])
  
  return m

def logLines(log):
  f = open(log, "r")
  return [x for x in f]

def get_params(lines, label):
  line = bestParams(lines)[label]

  line_params = line.split(',')
  c_ = int(line_params[0][line_params[0].index(': ') + 2:])
  fuzzifier_ = line_params[1].split('.')[-1][:-2]

  k_ = line_params[2][6:line_params[2].index('(')]
  param_k_ = float(line_params[2][line_params[2].index('(')+1:line_params[2].index(')')])
  return (c_, fuzzifier_, (k_, param_k_))

# for c in range(40):
#   print(c, get_params(logLines(LOG_COMPELTE_PATH), str(c)))

#params(logLines(LOG_COMPELTE_PATH)), len(dataDimension(logLines(LOG_COMPELTE_PATH)))
bestScores(logLines(LOG_COMPELTE_PATH))

{'0': '-0.025000000000000005',
 '1': '-0.025000000000000005',
 '10': '-0.025000000000000005',
 '11': '-0.025000000000000005',
 '12': '-0.025000000000000005',
 '13': '-0.025000000000000005',
 '14': '-0.025000000000000005',
 '15': '0.0',
 '16': '-0.007087614505952221',
 '17': '-0.025000000000000005',
 '18': '-0.025000000000000005',
 '19': '-0.025000000000000005',
 '2': '-0.025000000000000005',
 '20': '-0.025000000000000005',
 '21': '-0.025000000000000005',
 '22': '-0.025000000000000005',
 '23': '-0.025000000000000005',
 '24': '-0.025000000000000005',
 '25': '-0.025000000000000005',
 '26': '-0.025000000000000005',
 '27': '-0.025000000000000005',
 '28': '-0.025000000000000005',
 '29': '-0.025000000000000005',
 '3': '-0.025000000000000005',
 '30': '-0.025000000000000005',
 '31': '-0.025000000000000005',
 '32': '-0.025000000000000005',
 '33': '-0.025000000000000005',
 '34': '-0.025000000000000005',
 '35': '-0.025000000000000005',
 '36': '-0.025000000000000005',
 '37': '-0.025000000000000005'

# Plot utils

In [0]:
def show_scatter_plot(data, labels):
  #plt.figure(figsize=(15,15))
  for lab in range(40):
          plt.scatter(data[labels==lab, 0],
                      data[labels==lab, 1],
                      label=lab,
                      c=[np.random.rand(3,)],
                      marker="$" + str(lab) + "$",
                      s=200
                      )
  plt.show()

def gr_membership_contour(estimated_membership, data, labels, label, with_label=False, precision=.25):
  plt.figure(figsize=(15, 15))
  data_filtered = [list(elem) for elem, l in list(zip(data, labels)) if l == label]

  offsetX = (maxX(data_filtered) - minX(data_filtered))/2
  offsetY = (maxY(data_filtered) - minY(data_filtered))/2

  x = np.arange(minX(data_filtered) - offsetX, maxX(data_filtered) + offsetX, precision)
  y = np.arange(minY(data_filtered) - offsetY, maxY(data_filtered) + offsetY, precision)
  X, Y = np.meshgrid(x, y)
  zs = np.array([estimated_membership([[x, y]])
                  for x,y in zip(np.ravel(X), np.ravel(Y))])
  Z = zs.reshape(X.shape)

  membership_contour = plt.contour(X, Y, Z,
                                    levels=(.1, .3, .5, .7, .8, .9, .95), colors='k')
  
  if(with_label):
    plt.clabel(membership_contour, inline=1)

  plt.title("Membership contour on test {}".format(label))
  show_scatter_plot(data, labels)

  plt.show()


def minX(x): return min(x, key=lambda t: t[0])[0]

def maxX(x): return max(x, key=lambda t: t[0])[0]

def minY(x): return min(x, key=lambda t: t[1])[1]

def maxY(x): return max(x, key=lambda t: t[1])[1]

# Estimations

In [0]:
models = {}

for c in get_classes(olivetti_labels):
  c_, fuzzifier_, (kernel_, param_) = get_params(logLines(LOG_COMPELTE_PATH), str(c))
  print(c_, str2fuzzifier(fuzzifier_), str2kernel(kernel_)(param_))
  print(map_labels(y_train, c))
  models[c] = fl.FuzzyInductor(c=c_, fuzzifier=str2fuzzifier(fuzzifier_), k=str2kernel(kernel_)(param_)).fit(X_train, map_labels(y_train, c))
  print(models[c].score(X_test, map_labels(y_test, c)))

In [0]:
def get_predictions(models, dataset):
  return [models[model].predict(dataset) for model in models]

predictionsOnTrain = get_predictions(models, X_train)
predictionsOnTest = get_predictions(models, X_test)

In [0]:
l_test = []
l_train = []

if(l == None):
  l = []
  for model in models:
    l.append(models[model].predict(X_test))

In [0]:
zipped_predictions = list(zip(*l))
results = list(zip([sorted(enumerate(predict), reverse=True, key=lambda x: x[1])[0][0] for predict in zipped_predictions], y_test))

print("Result: {:.4f}".format(sum([1 if elem[0] == elem[1] else 0 for elem in results])/len(y_test)))
print("Different result: {}".format([elem for elem in results if elem[0] != elem[1]]))

Result: 0.8333
Different result: [(39, 4), (0, 7), (0, 28), (0, 2), (0, 6), (0, 3), (7, 9), (2, 25), (0, 2), (14, 12), (0, 22), (0, 34), (29, 20), (22, 25), (0, 31), (0, 12), (0, 27), (0, 3), (8, 22), (24, 12)]


# Plotted data sample




In [0]:
show_scatter_plot(X_test, y_test)
show_scatter_plot(X_train, y_train)

In [0]:
model_26.score(X_test, [1 if elem == 26 else 0 for elem in y_test])

-0.029848089107371578

In [0]:
show_scatter_plot(olivetti_data_tsne, olivetti_labels)

In [0]:
for model in models:
  gr_membership_contour(models[model].predict, X_test, y_test, model)

Output hidden; open in https://colab.research.google.com to view.

# Model selection

---

Here you can find all of the training done with ``` grid_search ``` or ``` nested_grid_search ```



In [0]:
def start(params, dimension, logger):
  lastAnalyzedClass_ = int(lastAnalyzedClass(logLines(LOG_COMPELTE_PATH)))
  dataDimension_ = lambda elem: int(dataDimension(logLines(LOG_COMPELTE_PATH))[elem])

  olivetti_data_tsne = TSNE(n_components=dimension).fit_transform(olivetti_data) if dimension < 4 else TSNE(n_components=dimension, method='exact').fit_transform(olivetti_data)
  from sklearn.decomposition import PCA
  pca_xd = PCA(n_components=dimension)
  olivetti_data_xd = pca_xd.fit_transform(olivetti_data)
  #grid_search(olivetti_data_tsne, olivetti_labels, params, 3, logger=None, y_needs_mapping=True, label=8)
  for c in get_classes(olivetti_labels):
    #if(c == 16 or c == 15):
    #if(c > lastAnalyzedClass_ or dataDimension_(c) != dimension):
      grid_search(olivetti_data_xd, olivetti_labels, params, dimension, data_reduction="p", logger=logger, y_needs_mapping=True, label=c)

In [0]:
params = {
    'c': [1, 1000],
    'k': [fl.kernel.GaussianKernel(sigma=.5), fl.kernel.GaussianKernel(sigma=.25)],
    'fuzzifier': [fl.fuzzifiers.LinearFuzzifier, fl.fuzzifiers.QuantileLinearPiecewiseFuzzifier]
}

dimension = 5

logger = log_init(fname="of_pca_5.log")

start(params, dimension, logger)

2020-03-19 10:21:44,404 - __main__ - INFO - (0) - Started GridSearchCV with: {'c': [1, 1000], 'k': [GaussianKernel(0.5), GaussianKernel(0.25)], 'fuzzifier': [<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>, <class 'fuzzylearn.fuzzifiers.QuantileLinearPiecewiseFuzzifier'>]}
2020-03-19 10:21:44,406 - __main__ - INFO - (0) - DATA_DIMENSION_: 5
100%|██████████| 100/100 [00:47<00:00,  2.20it/s]
2020-03-19 10:42:44,374 - __main__ - INFO - (0) - CV_RESULTS_: {'mean_fit_time': array([48.97986436, 48.75685644, 48.73014832, 48.99810719, 49.89457154,
       51.30523594, 52.33485937, 50.42329812]), 'std_fit_time': array([0.18742759, 0.22144363, 0.09856392, 0.23949496, 1.16338648,
       1.05238699, 0.89565077, 0.04915383]), 'mean_score_time': array([0.19485474, 0.2072947 , 0.18894196, 0.18662604, 0.20347683,
       0.21986556, 0.19915756, 0.20462139]), 'std_score_time': array([0.01577987, 0.01624531, 0.00475675, 0.00232792, 0.01587165,
       0.01095104, 0.01402106, 0.01413101]), 'param_c': masked

{'mean_fit_time': array([48.97986436, 48.75685644, 48.73014832, 48.99810719, 49.89457154,
       51.30523594, 52.33485937, 50.42329812]), 'std_fit_time': array([0.18742759, 0.22144363, 0.09856392, 0.23949496, 1.16338648,
       1.05238699, 0.89565077, 0.04915383]), 'mean_score_time': array([0.19485474, 0.2072947 , 0.18894196, 0.18662604, 0.20347683,
       0.21986556, 0.19915756, 0.20462139]), 'std_score_time': array([0.01577987, 0.01624531, 0.00475675, 0.00232792, 0.01587165,
       0.01095104, 0.01402106, 0.01413101]), 'param_c': masked_array(data=[1, 1, 1, 1, 1000, 1000, 1000, 1000],
             mask=[False, False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_fuzzifier': masked_array(data=[<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,
                   <class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,
                   <class 'fuzzylearn.fuzzifiers.QuantileLinearPiecewiseFuzzifier'>,
                   <class 'fuzzylearn.fu

100%|██████████| 100/100 [00:45<00:00,  2.14it/s]
2020-03-19 11:04:16,847 - __main__ - INFO - (1) - CV_RESULTS_: {'mean_fit_time': array([51.88668648, 52.7955029 , 54.25925406, 54.6331555 , 51.13577779,
       48.04612295, 48.51142724, 49.76864219]), 'std_fit_time': array([0.57948011, 0.1621372 , 1.10487539, 1.5555921 , 1.05065687,
       0.42176047, 0.72413808, 0.9380017 ]), 'mean_score_time': array([0.19887527, 0.22570062, 0.22586099, 0.1963381 , 0.20868436,
       0.19792589, 0.19127282, 0.20711637]), 'std_score_time': array([0.01053837, 0.00064456, 0.0172411 , 0.01487692, 0.01087421,
       0.01859845, 0.0137244 , 0.0120399 ]), 'param_c': masked_array(data=[1, 1, 1, 1, 1000, 1000, 1000, 1000],
             mask=[False, False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_fuzzifier': masked_array(data=[<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,
                   <class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,
            

{'mean_fit_time': array([51.88668648, 52.7955029 , 54.25925406, 54.6331555 , 51.13577779,
       48.04612295, 48.51142724, 49.76864219]), 'std_fit_time': array([0.57948011, 0.1621372 , 1.10487539, 1.5555921 , 1.05065687,
       0.42176047, 0.72413808, 0.9380017 ]), 'mean_score_time': array([0.19887527, 0.22570062, 0.22586099, 0.1963381 , 0.20868436,
       0.19792589, 0.19127282, 0.20711637]), 'std_score_time': array([0.01053837, 0.00064456, 0.0172411 , 0.01487692, 0.01087421,
       0.01859845, 0.0137244 , 0.0120399 ]), 'param_c': masked_array(data=[1, 1, 1, 1, 1000, 1000, 1000, 1000],
             mask=[False, False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_fuzzifier': masked_array(data=[<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,
                   <class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,
                   <class 'fuzzylearn.fuzzifiers.QuantileLinearPiecewiseFuzzifier'>,
                   <class 'fuzzylearn.fu

100%|██████████| 100/100 [00:45<00:00,  2.20it/s]
2020-03-19 11:24:48,956 - __main__ - INFO - (2) - CV_RESULTS_: {'mean_fit_time': array([49.91236345, 47.97488228, 50.38303693, 48.50958077, 48.24135534,
       48.20691117, 48.83051515, 49.07937948]), 'std_fit_time': array([1.24442881, 0.57582935, 0.63778804, 0.25430349, 0.06313503,
       0.38316759, 0.49423165, 0.42770959]), 'mean_score_time': array([0.19012292, 0.1985321 , 0.19901212, 0.19440126, 0.18463421,
       0.19108375, 0.18757335, 0.19915636]), 'std_score_time': array([0.00800528, 0.00450141, 0.00697628, 0.01072764, 0.00055399,
       0.00611828, 0.00682405, 0.01281119]), 'param_c': masked_array(data=[1, 1, 1, 1, 1000, 1000, 1000, 1000],
             mask=[False, False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_fuzzifier': masked_array(data=[<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,
                   <class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,
            

...

100%|██████████| 100/100 [00:45<00:00,  2.22it/s]
2020-03-19 16:00:05,080 - __main__ - INFO - (15) - CV_RESULTS_: {'mean_fit_time': array([47.97504775, 47.78499381, 48.15515224, 48.66808542, 48.41985536,
       48.56280963, 48.54212268, 48.65111772]), 'std_fit_time': array([0.06026693, 0.15153994, 0.17235579, 0.3225515 , 0.18617589,
       0.2457741 , 0.19290666, 0.48820569]), 'mean_score_time': array([0.18572291, 0.18349973, 0.18893337, 0.18823497, 0.19348009,
       0.19051115, 0.19525774, 0.19247039]), 'std_score_time': array([0.00189094, 0.00151523, 0.00510454, 0.00370234, 0.00538061,
       0.00348609, 0.00395066, 0.00302119]), 'param_c': masked_array(data=[1, 1, 1, 1, 1000, 1000, 1000, 1000],
             mask=[False, False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_fuzzifier': masked_array(data=[<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,
                   <class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,
           

{'mean_fit_time': array([47.97504775, 47.78499381, 48.15515224, 48.66808542, 48.41985536,
       48.56280963, 48.54212268, 48.65111772]), 'std_fit_time': array([0.06026693, 0.15153994, 0.17235579, 0.3225515 , 0.18617589,
       0.2457741 , 0.19290666, 0.48820569]), 'mean_score_time': array([0.18572291, 0.18349973, 0.18893337, 0.18823497, 0.19348009,
       0.19051115, 0.19525774, 0.19247039]), 'std_score_time': array([0.00189094, 0.00151523, 0.00510454, 0.00370234, 0.00538061,
       0.00348609, 0.00395066, 0.00302119]), 'param_c': masked_array(data=[1, 1, 1, 1, 1000, 1000, 1000, 1000],
             mask=[False, False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_fuzzifier': masked_array(data=[<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,
                   <class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,
                   <class 'fuzzylearn.fuzzifiers.QuantileLinearPiecewiseFuzzifier'>,
                   <class 'fuzzylearn.fu

100%|██████████| 100/100 [00:46<00:00,  2.18it/s]
2020-03-19 16:20:26,084 - __main__ - INFO - (16) - CV_RESULTS_: {'mean_fit_time': array([47.68166804, 47.98175693, 48.66797638, 48.59346875, 47.81091468,
       48.41710194, 48.69657461, 48.98985759]), 'std_fit_time': array([0.32102605, 0.13626375, 0.3720046 , 0.08813169, 0.06399668,
       0.28979154, 0.27421947, 0.20592694]), 'mean_score_time': array([0.19685388, 0.20569118, 0.18805583, 0.18727676, 0.19353278,
       0.19428102, 0.20340196, 0.18924109]), 'std_score_time': array([0.01065379, 0.01146292, 0.00271137, 0.00016078, 0.00457896,
       0.00666789, 0.00476445, 0.00285108]), 'param_c': masked_array(data=[1, 1, 1, 1, 1000, 1000, 1000, 1000],
             mask=[False, False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_fuzzifier': masked_array(data=[<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,
                   <class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,
           

{'mean_fit_time': array([47.68166804, 47.98175693, 48.66797638, 48.59346875, 47.81091468,
       48.41710194, 48.69657461, 48.98985759]), 'std_fit_time': array([0.32102605, 0.13626375, 0.3720046 , 0.08813169, 0.06399668,
       0.28979154, 0.27421947, 0.20592694]), 'mean_score_time': array([0.19685388, 0.20569118, 0.18805583, 0.18727676, 0.19353278,
       0.19428102, 0.20340196, 0.18924109]), 'std_score_time': array([0.01065379, 0.01146292, 0.00271137, 0.00016078, 0.00457896,
       0.00666789, 0.00476445, 0.00285108]), 'param_c': masked_array(data=[1, 1, 1, 1, 1000, 1000, 1000, 1000],
             mask=[False, False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_fuzzifier': masked_array(data=[<class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,
                   <class 'fuzzylearn.fuzzifiers.LinearFuzzifier'>,
                   <class 'fuzzylearn.fuzzifiers.QuantileLinearPiecewiseFuzzifier'>,
                   <class 'fuzzylearn.fu

 91%|█████████ | 91/100 [00:42<00:04,  2.21it/s]

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

In [0]:
lastAnalyzedClass(logLines(LOG_COMPELTE_PATH)) and dataDimension(logLines(LOG_COMPELTE_PATH))[int(lastAnalyzedClass(logLines(LOG_COMPELTE_PATH)))] != 5

In [0]:
dataDimension(logLines(LOG_COMPELTE_PATH))[int(lastAnalyzedClass(logLines(LOG_COMPELTE_PATH)))]

2

# AT&T dataset faces

In [0]:
import cv2

f = open(PROJ_COMPLETE_PATH + "AT&T_dataset_faces/dataset.txt", 'w')

for b in range(40):
  for a in range(10):
    data = cv2.imread(PROJ_COMPLETE_PATH + 'AT&T_dataset_faces/s' + str(b + 1) + '/' + str(a + 1) +'.pgm', -1)
    for line in data:
      for pixel in line:
        f.write(str(pixel) + " ")
    f.write('\n')

In [0]:
def atet_data():
  f_open = open(PROJ_COMPLETE_PATH + "AT&T_dataset_faces/dataset.txt", 'r')
  faces = []
  for line in f_open:
    faces.append([int(x) for x in line.split()])
  return np.asarray(faces)

def atet_labels():
  return np.ravel([[a] * 10 for a in range(40)])